# 2019 SMTO Uni/College Choice RF

Load the full 2019 SMTO dataframe:

In [1]:
import pandas as pd

full_df = pd.read_csv('../../Data/SMTO_2019/SMTO_2019_Complete_Input.csv')
full_df.head()

,Liv_Arr,Children,Cars,Income,Home_Zone,School_Name,Campus,Work,Licence,Mode,...,Dist.SHH,Dist.SHT,Dist.MI,Dist.SC,Dist.SG,Dist.YK,Dist.YG,Dist.RY,Dist.OC,PD
0,Live with family/parents,0.0,0.0,NaN,3851.0,Centennial College,Progress Campus,NW,False,Transit,...,4.821538,16.18253,19.03615,54.90925,32.749820,40.58064,43.982800,33.227630,31.525830,36
1,Live with family/parents,0.0,1.0,NaN,181.0,Centennial College,Morningside Campus,NW,False,Transit,...,43.017220,51.23784,22.93553,22.04248,8.990107,9.21471,8.279897,10.301550,10.302330,4
2,Live with family/parents,0.0,2.0,I don't know,1039.0,Centennial College,Progress Campus,NW,False,Transit,...,71.352090,81.92638,54.53982,13.64408,35.730980,40.61661,29.418650,35.055550,36.449570,20
3,NaN,NaN,NaN,NaN,191.0,Centennial College,Progress Campus,NW,False,NaN,...,40.200800,48.42141,20.66308,21.85741,5.659206,11.77978,8.357460,6.970653,6.971434,4
4,Live with family/parents,1.0,1.0,NaN,600.0,Centennial College,Progress Campus,NW,False,NaN,...,61.537790,69.75841,41.28113,11.49956,25.838390,18.81172,15.841280,25.162960,26.824060,16


Transform the complex income variable into a High/Low/Unknown variable:

In [2]:
def income_var(x):
    if x == 'Less than $ 14,999':
        return 'Low'
    elif x == '$ 15,000 - 29,999':
        return 'Low'
    elif x == '$ 30,000 - 39,999':
        return 'Low'
    elif x == '$ 40,000 - 49,999':
        return 'Low'
    elif x == '$ 50,000 - 59,999':
        return 'Low'
    elif x == '$ 60,000 - 69,999':
        return 'Low'
    elif x == '$ 70,000 - 79,999':
        return 'Low'
    elif x == '$ 80,000 - 89,999':
        return 'Low'
    elif x == '$ 90,000 - 99,999':
        return 'High'
    elif x == '$ 90,000 - 99,999':
        return 'High'
    elif x == '$ 100,000 - 124,999':
        return 'High'
    elif x == '$ 125,000 - 149,999':
        return 'High'
    elif x == '$ 150,000 - 199,999':
        return 'High'
    elif x == '$ 200,000 +':
        return 'High'
    else:
        return 'Unknown'
    
full_df['Income'] = full_df['Income'].apply(lambda x: income_var(x))

Split our dataframe into a training set (75% of the original dataframe) and a testing set. For now we will just work with our new training set called `df`.

In [3]:
# Split 2019 data into initial training and testing
df = full_df.sample(frac=0.75, random_state=0)
test_df = full_df.drop(df.index)
print(df.columns)

Index(['Liv_Arr', 'Children', 'Cars', 'Income', 'Home_Zone', 'School_Name',
       'Campus', 'Work', 'Licence', 'Mode', 'Age', 'Faculty', 'School_Type',
       'Family', 'Level', 'Status', 'Campus_Zone', 'School', 'Dist.CST',
       'Dist.CAS', 'Dist.CPR', 'Dist.CMO', 'Dist.CDV', 'Dist.CEG', 'Dist.CPI',
       'Dist.CDS', 'Dist.DOS', 'Dist.DWH', 'Dist.MCM', 'Dist.MCB', 'Dist.MOF',
       'Dist.MOS', 'Dist.MOI', 'Dist.OTD', 'Dist.OTN', 'Dist.SHD', 'Dist.SHH',
       'Dist.SHT', 'Dist.MI', 'Dist.SC', 'Dist.SG', 'Dist.YK', 'Dist.YG',
       'Dist.RY', 'Dist.OC', 'PD'],
      dtype='object')


#### Variables:

In [4]:
variable_list = ['Age','Children','Income', 'PD', 'Cars', 'Licence','Status']
#variable_list = ['PD', 'Age', 'Cars', 'Status']
df = pd.concat((df['School_Type'], df[variable_list],), axis = 1).dropna()

y = df['School_Type']
x = df[variable_list]
x = pd.concat((x, pd.get_dummies(x['Status'])), axis = 1)
del x['Status']
x = pd.concat((x, pd.get_dummies(x['Income'])), axis = 1)
del x['Income']
x.columns

Index(['Age', 'Children', 'PD', 'Cars', 'Licence', 'FT', 'PT', 'High', 'Low',
       'Unknown'],
      dtype='object')

In [5]:
x

,Age,Children,PD,Cars,Licence,FT,PT,High,Low,Unknown
11648,24.0,1.0,5,2.0,True,0,1,0,0,1
1794,18.0,0.0,16,2.0,True,1,0,0,0,1
575,22.0,0.0,20,1.0,True,1,0,0,1,0
4934,21.0,2.0,13,1.0,True,1,0,0,1,0
13545,18.0,0.0,16,1.0,True,1,0,0,1,0
5437,18.0,2.0,35,2.0,False,1,0,0,1,0
16683,18.0,0.0,33,2.0,True,1,0,1,0,0
12980,20.0,0.0,35,2.0,False,1,0,0,0,1
6711,34.0,0.0,12,0.0,False,0,1,0,0,1
14460,18.0,0.0,36,2.0,True,1,0,0,1,0


In [6]:
# Make sure x & y have the same length
print(x.shape)
print(y.shape)

(4466, 10)
(4466,)


#### Run Random Forest Model:

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.inspection import permutation_importance
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef

rf = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Accuracy\t", rf.score(X_test, y_test)) # Acc
print("PRF1 Mac\t", precision_recall_fscore_support(y_test, y_pred, average = 'macro')[:3]) # Rec = Bal Acc
print("Matthews\t", matthews_corrcoef(y_test, y_pred))

Accuracy	 0.8194029850746268
PRF1 Mac	 (0.6726602120792992, 0.6067401639437712, 0.6239987755726749)
Matthews	 0.27151264680607934


#### Feature Importance:

In [8]:
features = pd.DataFrame(index = X_test.columns)
features['FeatImportance'] = rf.feature_importances_
features.sort_values(by='FeatImportance' , inplace=True, ascending = False)
features

,FeatImportance
PD,0.397098
Age,0.289604
Cars,0.136571
Children,0.074664
Licence,0.038262
Low,0.017803
Unknown,0.015827
High,0.014122
FT,0.008146
PT,0.007904


#### Confusion Matrix:

In [9]:
# new df ('X_test_new') with Actual Choices and Predictions from X_Test
X_test_new = X_test
X_test_new = pd.concat([X_test_new, df['School_Type']], axis=1).reindex(X_test_new.index)

preds = rf.predict(X_test)
X_test_new['Preds'] = preds
#X_test_new.reset_index()

#probs = rf.predict_proba(X_test)
#probs = pd.DataFrame(probs)
#probs.rename(columns = {0:'College', 1: 'University'})



# Manual Confusion Matrix (NOT NEEDED NOW)
CM_hardmax = [[0,1],[2,3]]
CM_hardmax[0][0] = X_test_new[(X_test_new['School_Type'] == 'University') & (X_test_new['Preds'] == 'University')]['Preds'].count()
CM_hardmax[0][1] = X_test_new[(X_test_new['School_Type'] == 'University') & (X_test_new['Preds'] == 'College')]['Preds'].count()
CM_hardmax[1][0] = X_test_new[(X_test_new['School_Type'] == 'College') & (X_test_new['Preds'] == 'University')]['Preds'].count()
CM_hardmax[1][1] = X_test_new[(X_test_new['School_Type'] == 'College') & (X_test_new['Preds'] == 'College')]['Preds'].count()

CM_hardmax = pd.DataFrame(CM_hardmax)#.rename(columns = {0: 'Pred_Uni', 1: 'Pred_Col'}, rows = {0: 'Obs_Uni', 1: 'Obs_Col'} )

# Automatic Confusion Matrix
print('\nConfusion Matrix -- Hardmax:')
from sklearn.metrics import confusion_matrix
CM = pd.DataFrame(confusion_matrix(X_test_new['School_Type'].tolist(), X_test_new['Preds'].tolist()))
CM.rename(columns = {0: 'Pred_Col', 1: 'Pred_Uni'}, index = {0: 'Obs_Col', 1: 'Obs_Uni'} )


Confusion Matrix -- Hardmax:


,Pred_Col,Pred_Uni
Obs_Col,66,173
Obs_Uni,69,1032


... where 0 = College, and 1 = University

In [10]:
import seaborn as sn
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
sn.heatmap(pd.crosstab(X_test_new['School_Type'], X_test_new['Preds']), annot = True)
plt.show()

<Figure size 500x500 with 2 Axes>

In [11]:
from sklearn.metrics import f1_score

print('College f-1 Score: ' + str(f1_score(X_test_new['School_Type'].tolist(), X_test_new['Preds'].tolist(), average=None)[0]))

College f-1 Score: 0.35294117647058826
